# Perform and pickle cross-validation for all datasets

In [ ]:
%matplotlib notebook

import numpy as np
import pylab as plt
import seaborn as sns
import pickle
import matplotlib

import sparseRRR

In [ ]:
def preprocess(data, normalize='cpm'):
    X = data['counts'][:,data['mostVariableGenes']] / np.sum(data['counts'], axis=1)
    if normalize=='cpm':
        X *= 1e+6
    elif normalize=='median':
        X *= np.median(np.array(np.sum(data['counts'],axis=1)))
    X = np.array(X)
    X = np.log2(X + 1)
    X = X - np.mean(X, axis=0)
    X = X / np.std(X, axis=0)

    Y = data['ephys']
    Y = Y - np.mean(Y, axis=0)
    Y = Y / np.std(Y, axis=0)
    
    return (X,Y)

## The main cross-validation setup

In [ ]:
data = pickle.load(open('data/scala2020.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

In [ ]:
%%time

lambdas = np.concatenate((np.arange(.1,1,.1), np.arange(1,11)))
alphas = np.array([.25, .5, .75, 1])

cvresults = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=1, folds=10, alphas=alphas, lambdas=lambdas)

alphas = np.array([1])
ranks = np.arange(1, Y.shape[1]+1)

cvresults_rank = {}
for r in ranks:
    cvresults_rank[r] = sparseRRR.elastic_rrr_cv(X, Y, rank=r, reps=1, folds=10, alphas=alphas, lambdas=lambdas)
    
pickle.dump([cvresults, cvresults_rank], open('pickles/cvresults-scala2020.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/scala2019.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

In [ ]:
%%time

lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
alphas = [.25, .5, .75, 1]

cvresults = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=10, folds=10, alphas=alphas, lambdas=lambdas)

alphas = np.array([.5])
ranks = np.arange(1, Y.shape[1]+1)

cvresults_rank = {}
for r in ranks:
    cvresults_rank[r] = sparseRRR.elastic_rrr_cv(X, Y, lambdas=lambdas, alphas=alphas, reps=10, rank=r, folds=10)
    
pickle.dump([cvresults, cvresults_rank], open('pickles/cvresults-scala2019.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/cadwell2016.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

In [ ]:
%%time

lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
alphas = [.25, .5, .75, 1]

cvresults = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=10, folds=11, alphas=alphas, lambdas=lambdas)

alphas = np.array([.5])
ranks = np.arange(1, Y.shape[1]+1)

cvresults_rank = {}
for r in ranks:
    cvresults_rank[r] = sparseRRR.elastic_rrr_cv(X, Y, lambdas=lambdas, alphas=alphas, reps=10, rank=r, folds=11)
    
pickle.dump([cvresults, cvresults_rank], open('pickles/cvresults-cadwell2016.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/fuzik2016.pickle', 'rb'))
X,Y = preprocess(data, normalize='median')
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

In [ ]:
lambdas = np.arange(.1,6,.25)
alphas = np.array([.5])
ranks = np.array([1,2])

cvresults = {}
for r in ranks:
    cvresults[r] = sparseRRR.elastic_rrr_cv(X, Y, lambdas=lambdas, alphas=alphas, reps=10, rank=r, folds=10)
    
pickle.dump(cvresults, open('pickles/cvresults-fuzik2016.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/gouwens2020.pickle', 'rb'))
# Already preprocessed
X=data['counts'].toarray().astype('float64')
Y=data['ephys']
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape)

In [ ]:
%%time

lambdas = np.concatenate((np.arange(.3,1,.1), np.arange(1,11)))
alphas = np.array([.25, .5, .75, 1])

cvresults = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=1, folds=10, alphas=alphas, lambdas=lambdas)

alphas = np.array([1])
lambdas = lambdas = np.concatenate((np.arange(.3,1,.1), np.arange(1,11)))
ranks = np.arange(1, 17)

cvresults_rank = {}
for r in ranks:
    cvresults_rank[r] = sparseRRR.elastic_rrr_cv(X, Y, rank=r, reps=1, folds=10, alphas=alphas, lambdas=lambdas)
    
pickle.dump([cvresults, cvresults_rank], open('pickles/cvresults-gouwens2020.pickle', 'wb'))

## Nested CV

In [ ]:
data = pickle.load(open('data/scala2020.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape, '\n')

lambdas = np.concatenate([np.arange(.1,1,.1), np.arange(1,11)])
alphas = np.array([.25, .5, .75, 1])
%time sparseRRR.nested_cv(X, Y, lambdas, alphas)

In [ ]:
data = pickle.load(open('data/scala2019.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape, '\n')

lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
alphas = np.array([.25, .5, .75, 1])
%time sparseRRR.nested_cv(X, Y, lambdas, alphas)

In [ ]:
data = pickle.load(open('data/cadwell2016.pickle', 'rb'))
X,Y = preprocess(data)
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape, '\n')

lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
alphas = np.array([.25, .5, .75, 1])
%time sparseRRR.nested_cv(X, Y, lambdas, alphas)

In [ ]:
data = pickle.load(open('data/gouwens2020.pickle', 'rb'))
# Already preprocessed
X=data['counts'].toarray().astype('float64')
Y=data['ephys']
print('Shape of X:', X.shape, '\nShape of Y:', Y.shape, '\n')

lambdas = np.concatenate((np.arange(.3,1,.1), np.arange(1,11)))
alphas = np.array([.25, .5, .75, 1])
%time sparseRRR.nested_cv(X, Y, lambdas, alphas)

## Preprocessing variants

In [ ]:
import rnaseqTools

def cv_preprocessing_variants(data, filename, lambdas=[1,2,3], alpha=.5, reps=1, folds=10, n_genes=1000):
    X,Y = preprocess(data)

    alphas = np.array([alpha])
    cv1 = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=reps, 
                                   folds=folds, alphas=alphas, lambdas=lambdas)

    X = data['counts'][:,data['mostVariableGenes']] / np.sum(data['counts'], axis=1) * 1e+6
    X = np.array(X)
    X = np.log2(X + 1)
    X = X - np.mean(X, axis=0)
    cv2 = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=reps,
                                   folds=folds, alphas=alphas, lambdas=lambdas)

    X = data['counts'] / np.sum(data['counts'], axis=1) * 1e+6
    X = np.array(X)
    X = np.log2(X + 1)
    ind = np.sum(X, axis=0) > 0
    X = X - np.mean(X, axis=0)
    cv4 = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=reps,
                                   folds=folds, alphas=alphas, lambdas=lambdas)

    X = X[:, ind]
    X = X / np.std(X, axis=0)
    cv3 = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=reps,
                                   folds=folds, alphas=alphas, lambdas=lambdas)
    
    
    def cv_preprocess(Xtrain, Xtest, n_genes=n_genes):
        mostVariableGenes = rnaseqTools.geneSelection(Xtrain, n=n_genes, threshold=32, plot=False, verbose=0)

        X = Xtrain[:,mostVariableGenes] / np.sum(Xtrain, axis=1, keepdims=True) * 1e+6
        X = np.array(X)
        X = np.log2(X + 1)
        ind = np.sum(X, axis=0) > 0
        X = X[:, ind]
        mu = np.mean(X, axis=0)
        std = np.std(X, axis=0)
        X = (X - mu) / std
    
        Xt = Xtest[:,mostVariableGenes] / np.sum(Xtest, axis=1, keepdims=True) * 1e+6
        Xt = np.array(Xt)
        Xt = np.log2(Xt + 1)
        Xt = Xt[:, ind]
        Xt = (Xt - mu) / std
    
        return(X, Xt) 
    
    X = np.array(data['counts'].todense())
    cv5 = sparseRRR.elastic_rrr_cv(X, Y, rank=2, reps=reps,     
                                   folds=folds, alphas=alphas, lambdas=lambdas,
                                   preprocess = cv_preprocess)

    pickle.dump([cv1, cv2, cv3, cv4, cv5], open(filename, 'wb'))

In [ ]:
data = pickle.load(open('data/scala2020.pickle', 'rb'))
lambdas = np.concatenate((np.arange(.1,1,.1), np.arange(1,11)))
cv_preprocessing_variants(data, 'pickles/cvresults-scala2020-variants.pickle', 
                          lambdas=lambdas, alpha=1, reps=1, folds=10)

In [ ]:
data = pickle.load(open('data/scala2019.pickle', 'rb'))
lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
cv_preprocessing_variants(data, 'pickles/cvresults-scala2019-variants.pickle', 
                          lambdas=lambdas, alpha=.5, reps=10, folds=10)

In [ ]:
data = pickle.load(open('data/cadwell2016.pickle', 'rb'))
lambdas = np.concatenate([np.arange(.1,3,.1), np.arange(3,11)])
cv_preprocessing_variants(data, 'pickles/cvresults-cadwell2016-variants.pickle', 
                          lambdas=lambdas, alpha=.5, reps=10, folds=11, n_genes=3000)

## Witten and Suo comparisons

In [ ]:
# from rpy2.robjects.packages import importr
# utils = importr('utils')
# utils.install_packages('PMA')

In [ ]:
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

from rpy2.robjects.packages import importr
pma = importr("PMA")

def witten(X, Y, ncomps=1, lx=1):
    out = pma.CCA(X, Y, typex="standard", typez="standard", standardize=False, 
                  K=ncomps, penaltyx=lx)
    d = { key : out.rx2(key) for key in out.names }
    w = np.asarray(d['u'])
    v = np.asarray(d['v'])
    return (w,v)

def witten_cv(X, Y, reg_params, reps=1, folds=10, seed=42, ncomps=1):
    n = X.shape[0]
    testcorrs = np.zeros((folds, reps, len(reg_params), ncomps))
    nonzero = np.zeros((folds, reps, len(reg_params), ncomps))

    # CV repetitions
    np.random.seed(seed)
    for rep in range(reps):
        print('.', end='')
        ind = np.random.permutation(n)
        X = X[ind,:]
        Y = Y[ind,:]
        
        # CV folds
        for cvfold in range(folds):
            indtest  = np.arange(cvfold*int(n/folds), (cvfold+1)*int(n/folds))
            indtrain = np.setdiff1d(np.arange(n), indtest)
            Xtrain = np.copy(X[indtrain,:])
            Ytrain = np.copy(Y[indtrain,:])
            Xtest  = np.copy(X[indtest,:])
            Ytest  = np.copy(Y[indtest,:])
            
            # mean centering
            X_mean = np.mean(Xtrain, axis=0)
            Xtrain -= X_mean
            Xtest  -= X_mean
            Y_mean = np.mean(Ytrain, axis=0)
            Ytrain -= Y_mean
            Ytest  -= Y_mean

            # loop over regularization parameters
            for i,r in enumerate(reg_params):    
                vx,vy = witten(Xtrain, Ytrain, ncomps=ncomps, lx=r)
                
                if np.sum(np.sum(vx!=0,axis=0)==0)>0 or np.sum(np.sum(vy!=0,axis=0)==0)>0:
                    nonzero[cvfold, rep, i, :] = np.nan
                    continue
                
                for ncomp in range(ncomps):
                    testcorrs[cvfold, rep, i, ncomp] = np.corrcoef((Xtest @ vx[:,ncomp]).T, 
                                                                   (Ytest @ vy[:,ncomp]).T)[0,1]
                    nonzero[cvfold, rep, i, ncomp] = np.sum(vx[:,ncomp]!=0)
        
    print(' done')
    return testcorrs, nonzero

In [ ]:
data = pickle.load(open('data/scala2020.pickle', 'rb'))
X,Y = preprocess(data)

lx_scan = np.arange(.05, .20, .01)
%time corr_witten, nonz_witten = witten_cv(X, Y, lx_scan, ncomps=2, reps=1)

pickle.dump([corr_witten, nonz_witten], 
           open('pickles/cvresults-scala2020-witten.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/scala2019.pickle', 'rb'))
X,Y = preprocess(data)

lx_scan = np.arange(.05, .20, .01)
%time corr_witten, nonz_witten = witten_cv(X, Y, lx_scan, ncomps=2, reps=1)

pickle.dump([corr_witten, nonz_witten], 
           open('pickles/cvresults-scala2019-witten.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/cadwell2016.pickle', 'rb'))
X,Y = preprocess(data)

lx_scan = np.arange(.05, .20, .01)
%time corr_witten, nonz_witten = witten_cv(X, Y, lx_scan, ncomps=2, reps=1)

pickle.dump([corr_witten, nonz_witten], 
           open('pickles/cvresults-cadwell2016-witten.pickle', 'wb'))

In [ ]:
data = pickle.load(open('data/gouwens2020.pickle', 'rb'))
# Already preprocessed
X=data['counts'].toarray().astype('float64')
Y=data['ephys']

lx_scan = np.arange(.05, .20, .01)
%time corr_witten, nonz_witten = witten_cv(X, Y, lx_scan, ncomps=2, reps=1)

pickle.dump([corr_witten, nonz_witten], 
           open('pickles/cvresults-gouwens2020-witten.pickle', 'wb'))